# Langmuir Probe

Data analysis of the langmuir probe experiment.

Data is obtained

First, we import all the necessary modules

In [2]:
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit

import matplotlib.axes as axes
import matplotlib.pyplot as plt
# from google.colab import files

import pandas as pd
import glob
import os

import scipy


Now we define a custom function to load the data from the .txt files labview outputs

In [ ]:
def load_langmuir_data(file_path):
    """
    Load plasma data from file, extracting only the first section with 5 columns.

    Parameters:
    -----------
    file_path : str
        Path to the data file

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing the 5 columns of data
    """
    # Read file line by line
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Find the first empty line
    empty_line_index = -1
    for i, line in enumerate(lines):
        if line.strip() == '':
            empty_line_index = i
            break

    # Handle case where no empty line was found
    if empty_line_index == -1:
        print("Warning: No empty line found. Using all lines in the file.")
        data_lines = lines
    else:
        data_lines = lines[:empty_line_index]

    # Parse the data
    data = []
    skipped_lines = 0

    # Try to detect if we need to handle different line formats
    if len(data_lines) > 0:
        sample_line = data_lines[0].strip()
        if sample_line.endswith(','):
            print("Detected lines ending with commas. Will remove trailing commas.")
        else:
            print("Lines do not end with commas.")

    for line in data_lines:
        # Remove trailing comma if present
        clean_line = line.rstrip(',\r\n')
        # Split by comma
        values = clean_line.split(',')

        # Filter out empty strings and convert to float
        # If there's an empty value, replace with NaN
        float_values = []
        for val in values:
            if val.strip():  # Check if the value is not just whitespace
                float_values.append(float(val))
            else:
                float_values.append(float('nan'))

        # Only take the first 5 columns (pad with NaN if fewer than 5)
        while len(float_values) < 5:
            float_values.append(float('nan'))

        # Skip lines that would cause problems (all NaN or empty)
        if not all(np.isnan(val) for val in float_values[:5]):
            data.append(float_values[:5])
        else:
            skipped_lines += 1

    # Create DataFrame
    df = pd.DataFrame(data, columns=[
        'Voltage (V)',  # Based on the first column values
        'Current (A)',  # Small values in scientific notation
        'Unknown1',     # Large values with scientific notation
        'Unknown2',     # Values around 4130-4190
        'Unknown3'      # Constant values at 21508
    ])

    if skipped_lines > 0:
        print(f"Warning: Skipped {skipped_lines} lines that were empty or could not be parsed.")

    return df

In [ ]:
DATA_DIRECTORY = "/content/physics191_plasma/athalia and jorge's data/langmuir/"
